# Petition Results - Automated Download and Reporting

Created by Michael George (AKA Logiqx)

## Import Libraries

In [100]:
import urllib.request
import ssl
import json
import os
import datetime

## Process History

In [101]:
def processHistory(attributes, f):
    '''Process history for a petition'''

    def getDateTime(dt):
        dtSplit = dt.split('T')
        dateStr = dtSplit[0]
        timeStr = dtSplit[1].split('.')[0]
        return '%s | %s' % (dateStr, timeStr)

    f.write('### History\n')
    f.write('| Event | Date | Time |\n')
    f.write('| - | - | - |\n')
    f.write('| Updated | %s |\n' % getDateTime(attributes['updated_at']))
    
    if (attributes['debate_threshold_reached_at']):
        f.write('| Debate Threshold Reached | %s |\n' % getDateTime(attributes['debate_threshold_reached_at']))
    if (attributes['response_threshold_reached_at']):
        f.write('| Response Threshold Reached | %s |\n' % getDateTime(attributes['response_threshold_reached_at']))
        
    f.write('| Opened | %s |\n' % getDateTime(attributes['opened_at']))
    
    if (attributes['moderation_threshold_reached_at']):
        f.write('| Moderation Threshold Reached | %s |\n' % getDateTime(attributes['moderation_threshold_reached_at']))
        
    f.write('| Created | %s |\n' % getDateTime(attributes['created_at']))

## Process Countries

In [102]:
def processCountries(attributes, f):
    '''Process all of the countries for a petition'''

    # Count signature worldwide
    worldTotal = attributes['signature_count']

    # Count signatures for all of the countries
    total = 0
    for country in attributes['signatures_by_country']:
        total += country['signature_count']

    # Ensure that all signatures are accounted for by creating an 'Unknown' category
    diff = worldTotal - total
    if diff:
        if diff / worldTotal > 0.1:
            print('Missing country: %0.2f%%' % (diff / worldTotal))
        unknown = {'name': 'Unknown', 'code': 'ZZ', 'signature_count':  diff}
        attributes['signatures_by_country'].append(unknown)
        total += diff

    # List all of the countries sorted by signature count
    countries = sorted(attributes['signatures_by_country'], key=lambda country: country['signature_count'], reverse=True)

    f.write('### Signatures by Country\n')
    f.write('| # | Country | Signatures | % |\n')
    f.write('| - | - | -: | -: |\n')
    
    for i in range(len(countries)):
        country = countries[i]
        pct = 100 * country['signature_count'] / total
        f.write('| %d | %s | %s | %0.2f |\n' % (i + 1, country['name'], '{:,}'.format(country['signature_count']), pct))

## Process Constituencies

In [103]:
def processConstituencies(attributes, f):
    '''Process all of the constituencies for a petition'''

    # Count signatures for the UK (ISO code GB)
    ukTotal = 0
    for country in attributes['signatures_by_country']:
        if country['code'] == 'GB':
            ukTotal = country['signature_count']
    
    # Count signatures for all of the constituencies
    total = 0
    for constituency in attributes['signatures_by_constituency']:
        total += constituency['signature_count']
    
    # Ensure that all signatures are accounted for by creating an 'Unknown' category
    diff = ukTotal - total
    if diff:
        if diff / ukTotal > 0.1:
            print('Missing constituency: %0.2f%%' % (diff / ukTotal))
        unknown = {'name': 'Unknown', 'ons_code': 'Z99999999', 'mp': 'Unknown', 'signature_count':  diff}
        attributes['signatures_by_constituency'].append(unknown)
        total += diff

    # List all of the constituencies sorted by signature count
    constituencies = sorted(attributes['signatures_by_constituency'], key=lambda constituency: constituency['signature_count'], reverse=True)
    
    f.write('### Signatures by Constituency\n')
    f.write('| # | Constituency | MP | Signatures | % |\n')
    f.write('| - | - | - | -: | -: |\n')
    
    for i in range(len(constituencies)):
        constituency = constituencies[i]
        pct = 100 * constituency['signature_count'] / total
        f.write('| %d | %s | %s | %s | %0.2f |\n' % (i + 1, constituency['name'], constituency['mp'],
            '{:,}'.format(constituency['signature_count']), pct))

## Process Petition

In [104]:
def processPetition(petition):
    '''Process a single petition'''
    
    # Specify the URL
    url = petition['links']['self']
    
    # Do not verify certicates
    context = ssl._create_unverified_context()

    # Open the URL and retrieve the JSON
    handle = urllib.request.urlopen(url, context=context)

    # Parse JSON
    petition = json.load(handle)
    
    # Everything of interest is under "attributes"
    petitionData = petition['data']
    petitionId = petitionData['id']
    attributes = petitionData['attributes']
    
    # Remove newlines from the action / title
    attributes['action'] = attributes['action'].replace('\r\n',' ')
    
    # Progress message
    print('Processing %s...' % attributes['action'])

    # Write JSON to local file
    with open("../data/%d.json" % petitionId, 'w') as f:
        f.write(json.dumps(petition, indent=4))

    # Write summary to local file
    with open("../reports/%d.md" % petitionId, 'w') as f:
        f.write('# %s\n' % attributes['action'])
        f.write('%s\n\n' % attributes['background'])
        f.write('Data taken from [%s](%s)\n\n' % (petition['links']['self'], petition['links']['self'].replace('.json','')))
        f.write('**%s signatures**\n' % '{:,}'.format(attributes['signature_count']))

        processHistory(attributes, f)
        processCountries(attributes, f)
        processConstituencies(attributes, f)
        
    return petition

## Process Petitions

In [105]:
def processPetitions():
    '''Process the top petitions'''
    
    # Specify the URL
    url = "https://petition.parliament.uk/petitions.json?state=open"

    # Do not verify certicates
    context = ssl._create_unverified_context()

    # Open the URL and retrieve the JSON
    handle = urllib.request.urlopen(url, context=context)

    # Parse JSON
    index = json.load(handle)
    
    # Write JSON to local file
    with open("../data/index.json", 'w') as f:
        f.write(json.dumps(index, indent=4))
    
    # Write summary to local file
    with open("../README.md", 'w') as f:
        topX = len(index['data'])
        now = datetime.datetime.now()
        
        f.write('# Top %d Petitions\n' % topX)
        f.write('Data taken from [UK Government and Parliament](%s)\n\n' % url.replace('.json', ''))
        f.write('Reports created using a simple [IPython Notebook](python/Petition%20Reports.ipynb)\n\n')
        f.write('Last refreshed %s\n\n' % now.strftime("%Y-%m-%d %H:%M:%S"))
        f.write('| Petition Name | Total Signatures | UK Signatures | Overseas Signatures | Unknown Signatures | UK Signatures % | Overseas Signatures % | Unknown Signatures % |\n')
        f.write('| - | -: | -: | -: | -: | -: | -: | -: |\n')

        # Process the top X petitions
        for i in range(topX):
            petition = index['data'][i]
            petition = processPetition(petition)
            attributes = petition['data']['attributes']

            total = attributes['signature_count']
            ukTotal = unknownTotal = 0
            for country in attributes['signatures_by_country']:
                if country['code'] == 'GB':
                    ukTotal = country['signature_count']
                if country['code'] == 'ZZ':
                    unknownTotal = country['signature_count']
            overseasTotal = total - ukTotal - unknownTotal

            f.write('| [%s](reports/%s.md) | %s | %s | %s | %s | %0.2f | %0.2f | %0.2f |\n' % (
                attributes['action'], petition['data']['id'], '{:,}'.format(total),
                ('{:,}'.format(ukTotal)), ('{:,}'.format(overseasTotal)), ('{:,}'.format(unknownTotal)),
                100 * ukTotal / total, 100 * overseasTotal / total, 100 * unknownTotal / total
            ))

processPetitions()

Processing Revoke Article 50 and remain in the EU....
Processing Ban all ISIS members from returning to UK...
Processing Leave the EU without a deal in March 2019....
Processing Ban the sale of fireworks to the public. Displays for licenced venues only....
Processing School should start at 10am as teenagers are too tired...
Processing Make 'netting' hedgerows to prevent birds from nesting a criminal offence....
Processing Revoke Art.50 if there is no Brexit plan by the 25 of February...
Processing Grant a People's Vote if Parliament rejects the EU Withdrawal Agreement...
Processing Parliament must honour the Referendum result. Leave deal or no deal 29/03/19...
Processing Violet-Grace’s Law - Life sentences for Death by Dangerous Driving...
Processing Immunity for soldiers who worked in Northern Ireland. ...
Processing Ban the use of all non-recyclable and unsustainable food packaging...
Processing The UK should not agree the UN's Global Compact for Migration...
Processing Hold a second